In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Conv2D  
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
import tensorflow_addons as tfa
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.layers.core import Activation
from keras.callbacks import ReduceLROnPlateau
from datetime import datetime
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from PIL import Image
import cv2
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

## Padding the Image and Change the Color Channel

In [ ]:
def load(disease):
  dis_dir = os.path.join('C:/Users/Azyain/Downloads/archive (10)/RiceLeafs/train', disease) #Ganti make dir lu yang isinya folder per-class
  files = os.listdir(dis_dir)
  paths = [os.path.join(dis_dir, file) for file in files]
  imgs = [read(file) for file in paths]
  return np.array(imgs)

def normalization(image):
    height, width, _ = image.shape
    norm_size = max(height, width)
    add_h = norm_size - height
    add_w = norm_size - width 
    start_add_h = add_h//2
    start_add_w = add_w//2
    img = np.full((norm_size, norm_size, 3), 255, dtype=np.uint8)
    img[start_add_h:start_add_h + height, start_add_w:start_add_w + width, :] = image
    return img

def contrast_processing(image):
    img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
    img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
    return img

def preprocessing(image):
    img = normalization(image)
    img = cv2.resize(img, (300, 300), interpolation = cv2.INTER_AREA)
    img = contrast_processing(img)
    return np.array(img)

def read(image_file):
  img = cv2.imread(image_file)
  img = preprocessing(img)
  return np.array(img)
     

In [ ]:
diseases=os.listdir('C:/Users/Azyain/Downloads/archive (10)/RiceLeafs/train')#Ganti make dir lu yang isinya folder per-class

In [ ]:
diseases

## Generate Image

In [ ]:
disease_imgs = {}

for disease in diseases:
  disease_imgs[disease] = load('C:/Users/Azyain/Downloads/archive (10)/RiceLeafs/train', disease) #Ganti make dir lu yang isinya folder per-class
  print(disease, "Finisih with", len(disease_imgs[disease]), "data") 

disease_imgs.keys()

In [ ]:
datagen = ImageDataGenerator(
            rotation_range=30, #Ganti sesuai kebutuhan(Saran di 20-40)
            horizontal_flip=True, 
            vertical_flip=True, 
            brightness_range=[0.75,1.25],#Ganti sesuai kebutuhan
            width_shift_range=0.15,#Ganti sesuai kebutuhan
            height_shift_range=0.15, #Ganti sesuai kebutuhan
            zoom_range=0.15, #Ganti sesuai kebutuhan
          )

def generate_image(datagen, data, amount, savepath=None):
  it = datagen.flow(
        data, 
        batch_size=1, 
        save_to_dir=savepath,
        save_prefix='',
        save_format='png',
      )
  
  for i in range(amount):
    batch = it.next()
    image = batch[0].astype('uint8')

In [ ]:
amount = 1000 #ganti sesuai kebutuhan, gw taro di 1000

for idx, disease in enumerate(diseases):
    save_path = os.path.join('C:/Users/Azyain/Downloads/archive (10)/RiceLeafs/train-preprocess', disease) #Buat folder train-preprocess manual terus copy pathnya langsung disitu
    os.makedirs(save_path, exist_ok=True)
    generate_image(datagen, disease_imgs[disease], amount, save_path)
    print(f"{disease} : {len(os.listdir(save_path))}")